In [49]:
import os
import time

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision as tv
import torchvision.datasets as datasets
from art.attacks.evasion import BasicIterativeMethod, CarliniL2Method, FastGradientMethod
from art.estimators.classification import PyTorchClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV, ParameterGrid, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import RadiusNeighborsClassifier, NearestNeighbors
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

from util import get_correct_examples, dataset2tensor

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device: {}'.format(device))

n_threads = os.cpu_count()
print('CPU threads: {}'.format(n_threads))

Device: cuda
CPU threads: 24


In [4]:
PATH = 'data'
BATCH_SIZE = 128
EPOCHS = 5

In [5]:
# Fetch dataset
transforms = tv.transforms.Compose([tv.transforms.ToTensor()])
dataset_train = datasets.MNIST(PATH, train=True, download=True, transform=transforms)
dataset_test = datasets.MNIST(PATH, train=False, download=True, transform=transforms)

dataloader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
dataloader_test = DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=True)

print(dataset_train.data.size())
print(dataset_test.data.size())

## Train point-based classifier

In [6]:
# Create Neural Network model
model = nn.Sequential(
    nn.Conv2d(1, 32, 3, 1),
    nn.ReLU(),
    nn.Conv2d(32, 64, 3, 1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(9216, 200),
    nn.ReLU(),
    nn.Linear(200, 10),
    nn.LogSoftmax(dim=1)
)
model.to(device)
print('Number of layers: {}'.format(len(list(model.children()))))

Number of layers: 10


In [7]:
def train(model, loader, loss, optimizer):
    model.train()
    total_loss = 0.
    corrects = 0.
    
    for x, y in loader:
        x = x.to(device)
        y = y.to(device)
        batch_size = x.size(0)
        
        optimizer.zero_grad()
        outputs = model(x)
        l = loss(outputs, y)
        l.backward()
        optimizer.step()

        # for display
        total_loss += l.item() * batch_size
        preds = outputs.max(1, keepdim=True)[1]
        corrects += preds.eq(y.view_as(preds)).sum().item()
    
    n = len(loader.dataset)
    total_loss = total_loss / n
    accuracy = corrects / n
    return total_loss, accuracy

In [8]:
def validate(model, loader, loss):
    model.eval()
    total_loss = 0.
    corrects = 0.
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            batch_size = x.size(0)
            outputs = model(x)
            l = loss(outputs, y)
            total_loss += l.item() * batch_size
            preds = outputs.max(1, keepdim=True)[1]
            corrects += preds.eq(y.view_as(preds)).sum().item()
    
    n = len(loader.dataset)
    total_loss = total_loss / n
    accuracy = corrects / n
    return total_loss, accuracy

In [9]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
loss=nn.NLLLoss()

In [10]:
since = time.time()
for epoch in range(EPOCHS):
    start = time.time()
    tr_loss, tr_acc = train(model, dataloader_train, loss, optimizer)
    va_loss, va_acc = validate(model, dataloader_test, loss)
    
    time_elapsed = time.time() - start
    print(('[{:2d}] {:.0f}m {:.1f}s Train Loss: {:.4f} Accuracy: {:.4f}%, ' +
        'Test Loss: {:.4f} Accuracy: {:.4f}%').format(
            epoch+1, time_elapsed // 60, time_elapsed % 60,
            tr_loss, tr_acc*100.,
            va_loss, va_acc*100.))
    
time_elapsed = time.time() - since
print('Total run time: {:.0f}m {:.1f}s'.format(
    time_elapsed // 60,
    time_elapsed % 60))

[ 1] 0m 4.4s Train Loss: 0.4440 Accuracy: 87.4017%, Test Loss: 0.1358 Accuracy: 95.8500%
[ 2] 0m 4.4s Train Loss: 0.1181 Accuracy: 96.3983%, Test Loss: 0.0859 Accuracy: 97.3300%
[ 3] 0m 4.3s Train Loss: 0.0714 Accuracy: 97.8183%, Test Loss: 0.0584 Accuracy: 98.1500%
[ 4] 0m 4.5s Train Loss: 0.0487 Accuracy: 98.4933%, Test Loss: 0.0519 Accuracy: 98.3900%
[ 5] 0m 4.4s Train Loss: 0.0376 Accuracy: 98.8233%, Test Loss: 0.0431 Accuracy: 98.7200%
Total run time: 0m 22.0s


In [11]:
print('Training set: {}'.format(len(dataset_train)))
tensor_train_X, tensor_train_y = get_correct_examples(model, dataset_train, device=device, return_tensor=True)
dataset_train_perfect = TensorDataset(tensor_train_X, tensor_train_y)
dataloader_train_perfect = DataLoader(dataset_train_perfect, batch_size=512, shuffle=True)
_, acc = validate(model, dataloader_train_perfect, loss)
print('Accuracy on {} filtered training examples: {:.4f}%'.format(len(dataloader_train_perfect.dataset), acc*100))

print('Test set: {}'.format(len(dataset_test)))
tensor_test_X, tensor_test_y = get_correct_examples(model, dataset_test, device=device, return_tensor=True)
dataset_test_perfect = TensorDataset(tensor_test_X, tensor_test_y)
dataloader_test_perfect = DataLoader(dataset_test_perfect, batch_size=512, shuffle=True)
_, acc = validate(model, dataloader_test_perfect, loss)
print('Accuracy on {} filtered test examples: {:.4f}%'.format(len(dataloader_test_perfect.dataset), acc*100))

Training set: 60000
Accuracy on 59531 filtered training examples: 100.0000%
Test set: 10000
Accuracy on 9872 filtered test examples: 100.0000%


In [12]:
N_ADV = 2000

In [13]:
classifier = PyTorchClassifier(
    model=model, 
    loss=loss, 
    input_shape=(1, 28, 28), 
    optimizer=optimizer,
    nb_classes=10,
    clip_values=(0.0, 1.0),
    device_type=device
)

attack = FastGradientMethod(estimator=classifier, eps=0.2)
# attack = BasicIterativeMethod(estimator=classifier, eps=0.2)

In [14]:
# Prepare dataset for adversarial examples
n = len(dataset_test_perfect)
indices = torch.randperm(n)[:N_ADV]

pt_subset_X = tensor_test_X[indices]  # PyTorch Tensor
pt_subset_y = tensor_test_y[indices]

subset_X = pt_subset_X.cpu().detach().numpy()
subset_y = pt_subset_y.cpu().detach().numpy()

In [15]:
# Create adversarial examples
subset_pred = np.argmax(classifier.predict(subset_X), axis=1)
accuracy = np.sum(subset_pred == subset_y) / float(len(subset_pred))
print("Model accuracy on clean examples: {:.4f}%".format(accuracy * 100))

# Generate adversarial examples
subset_adv = attack.generate(x=subset_X)
subset_pred = np.argmax(classifier.predict(subset_adv), axis=1)

accuracy = np.sum(subset_pred == subset_y) / float(len(subset_pred))
print("Model accuracy on adversarial examples: {:.4f}%".format(accuracy * 100))

Model accuracy on clean examples: 100.0000%
Model accuracy on adversarial examples: 40.7000%


## Region-based classification

In [51]:
from sklearn import neighbors

In [37]:
# From PyTorch dataset to Numpy array
pt_X_train, pt_y_train = dataset2tensor(dataset_train)
X_train = pt_X_train.cpu().detach().numpy()
y_train = pt_y_train.cpu().detach().numpy()

# Reshape 2D images into 1D vector
X_train = X_train.reshape((X_train.shape[0], -1))

# Split model training set into training set and validation set
X_rc_train, X_rc_val, y_rc_train, y_rc_val = train_test_split(X_train, y_train, test_size=5000)

print(X_rc_train.shape)
print(X_rc_val.shape)

(55000, 784)
(5000, 784)


In [45]:
# rnn = RadiusNeighborsClassifier(radius=0.02, weights='uniform', p=2)
# rnn.fit(X_rc_train, y_rc_train)

RadiusNeighborsClassifier(radius=0.02)

In [80]:
# chebyshev: L-infinity norm
# manhattan: L1 norm
# euclidean: L2 norm

nn = NearestNeighbors(n_jobs=-1, metric='chebyshev')
nn.fit(X_rc_train, y_rc_train)

NearestNeighbors(metric='chebyshev', n_jobs=-1)

In [99]:
nn.kneighbors([X_rc_val[0]], n_neighbors=100)

(array([[0.69803923, 0.79607843, 0.79607844, 0.86666666, 0.87058824,
         0.8745098 , 0.89803922, 0.90196078, 0.90196079, 0.90588236,
         0.91764706, 0.92941177, 0.93333333, 0.94117647, 0.94117647,
         0.94509804, 0.95294118, 0.95686275, 0.96078431, 0.96078431,
         0.96470588, 0.96470588, 0.96862745, 0.96862745, 0.96862745,
         0.97254902, 0.97254902, 0.97647059, 0.97647059, 0.97647059,
         0.97647059, 0.97647059, 0.97647059, 0.97647059, 0.97647059,
         0.97647059, 0.97647059, 0.97647059, 0.97647059, 0.97647059,
         0.97647059, 0.97647059, 0.97647059, 0.97647059, 0.97647059,
         0.97647059, 0.97647059, 0.97647059, 0.97647059, 0.98039216,
         0.98039216, 0.98039216, 0.98431373, 0.98431373, 0.98431373,
         0.98823529, 0.98823529, 0.98823529, 0.98823529, 0.98823529,
         0.98823529, 0.98823529, 0.98823529, 0.98823529, 0.98823529,
         0.98823529, 0.98823529, 0.98823529, 0.98823529, 0.98823529,
         0.98823529, 0.98823529, 0

In [100]:
np.sum(np.abs(X_rc_train[53112] - X_rc_val[0]))

20.533333

In [102]:
np.sqrt(np.sum((X_rc_train[53112] - X_rc_val[0])**2))

2.742458

In [103]:
np.max(np.abs(X_rc_train[53112] - X_rc_val[0]))

0.69803923

In [117]:
print(y_rc_train[53112], y_rc_val[0], y_rc_train[12921])

7 7 7


In [98]:
np.where(np.all(np.abs(X_rc_train - X_rc_val[0]) < 0.3, axis=1))

(array([], dtype=int64),)

In [104]:
X_min = X_rc_val[0] - 0.3
X_max = X_rc_val[0] + 0.3

In [107]:
np.where(np.all(X_rc_train >= X_min and X_rc_train <= X_max))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [113]:
np.where(np.all(X_rc_train >= X_min, axis=1))

(array([17131, 42756]),)

In [114]:
np.where(np.all(X_rc_train <= X_max, axis=1))

(array([], dtype=int64),)

In [116]:
print(y_rc_train[17131], y_rc_train[42756])

7 7
